In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
#  !pip install tf-nightly
#except Exception:
 # pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
import nltk

print(tf.__version__)

2022-10-18 22:08:59.296558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 22:08:59.436549: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-18 22:08:59.490603: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-18 22:09:00.069550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

2.10.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-10-18 22:09:01--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::ac43:4695, 2606:4700:20::681a:321, 2606:4700:20::681a:221, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::ac43:4695|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  1.52MB/s    in 0.2s    

2022-10-18 22:09:01 (1.52 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2022-10-18 22:09:02--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::ac43:4695, 2606:4700:20::681a:321, 2606:4700:20::681a:221, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org

In [3]:
df_train = pd.read_csv(train_file_path, sep="\t", header=None, names=['y', 'x'])
df_train.head()

,y,x
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [4]:
df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=['y', 'x'])
df_test.head()

,y,x
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [5]:
y_train = df_train['y'].astype('category').cat.codes
y_test  = df_test['y'].astype('category').cat.codes
y_train[:5]

0    0
1    0
2    0
3    0
4    0
dtype: int8

In [6]:
stopwords_txt = set(stopwords.words('english'))


In [7]:
l = WordNetLemmatizer()

def clean_txt(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([l.lemmatize(word) for word in txt.split()
                    if not word in stopwords_txt])
    txt = txt.lower()
    return txt

In [8]:
X_train = df_train['x'].apply(lambda x: clean_txt(x))


In [9]:
max_words = 1000
max_len = 500
t = Tokenizer(num_words=max_words)
t.fit_on_texts(X_train)
sequences = t.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen=max_len)


In [10]:
inp = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_words, 50, input_length=max_len)(inp)
x = tf.keras.layers.LSTM(64)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=inp, outputs=x)
model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 embedding (Embedding)       (None, 500, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         


2022-10-18 22:09:03.311733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 22:09:03.346046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 22:09:03.346140: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 22:09:03.346546: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 96,337
Trainable params: 96,337
Non-trainable params: 0
_________________________________________________________________


In [19]:
r = model.fit(sequences_matrix, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2,
              )

Epoch 1/10
27/27 [==============================] - 0s 17ms/step - loss: 0.0718 - accuracy: 0.9913 - val_loss: 0.1354 - val_accuracy: 0.9880
Epoch 2/10
27/27 [==============================] - 0s 15ms/step - loss: 0.0740 - accuracy: 0.9838 - val_loss: 0.1185 - val_accuracy: 0.9904
Epoch 3/10
27/27 [==============================] - 0s 15ms/step - loss: 0.0537 - accuracy: 0.9934 - val_loss: 0.1018 - val_accuracy: 0.9916
Epoch 4/10
27/27 [==============================] - 0s 15ms/step - loss: 0.0469 - accuracy: 0.9952 - val_loss: 0.1315 - val_accuracy: 0.9916
Epoch 5/10
27/27 [==============================] - 0s 15ms/step - loss: 0.0500 - accuracy: 0.9952 - val_loss: 0.1191 - val_accuracy: 0.9892
Epoch 6/10
27/27 [==============================] - 0s 15ms/step - loss: 0.0528 - accuracy: 0.9955 - val_loss: 0.1320 - val_accuracy: 0.9904
Epoch 7/10
27/27 [==============================] - 0s 16ms/step - loss: 0.0488 - accuracy: 0.9964 - val_loss: 0.1228 - val_accuracy: 0.9892
Epoch 8/10
27

In [24]:
def preprocessing(X):
    x = X.apply(lambda x: clean_txt(x))
    x = t.texts_to_sequences(x)
    return pad_sequences(x, maxlen=max_len)
evall = model.evaluate(preprocessing(df_test['x']), y_test)


44/44 [==============================] - 0s 5ms/step - loss: 0.1499 - accuracy: 0.9828


In [25]:
print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evall[0], evall[1]))

Loss: 0.150, Accuracy: 0.983


In [26]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 12ms/step
(0.0, 'ham')


In [27]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 11ms/step
You passed the challenge. Great job!
